### Zigzag functions 

In [1]:
#import talib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime
import time

import matplotlib.dates as mdates
from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator
from datetime import timedelta
#from mpl_finance import candlestick2_ohlc, candlestick_ohlc
import matplotlib.ticker as ticker

from scipy.spatial import distance 
from sklearn.utils import validation
from sklearn.metrics import pairwise
from scipy.sparse import issparse

In [2]:
def get_zigzag(df, InpDepth, InpDeviation, InpBackstep, pip): 
  
    bars=len(df)  
    high=list(df.high)
    low=list(df.low)
    #high=[0 if x==10 else x for x in high]

    prev_calculated=0  
    ExtLevel=3

    i=0
    limit=0
    counterZ=0
    whatlookfor=0

    back=0
    pos=0
    lasthighpos=0
    lastlowpos=0

    curlow=0
    curhigh=0
    lasthigh=0
    lastlow=0

    point = pip # 0.0001 # pips

    if(InpBackstep>=InpDepth):
        print("Backstep cannot be greater or equal to Depth")
        pass

    #check for history and inputs
    if(bars<InpDepth or InpBackstep>=InpDepth):
        return 0
        pass

    # first calculations
    if prev_calculated==0:
        ExtZigzagBuffer=[0]*bars
        ExtHighBuffer=[0]*bars
        ExtLowBuffer=[0]*bars
        limit=bars-InpDepth
    else: 
        # find first extremum in the depth ExtLevel or 100 last bars
        for i in range(100): 
            for counterZ in range(ExtLevel): 
                if ExtZigzagBuffer[i]!=0: 
                    counterZ+=1
                i+=1

        # no extremum found - recounting all from begin
        if counterZ==0:
            ExtZigzagBuffer=[0]*bars
            ExtHighBuffer=[0]*bars
            ExtLowBuffer=[0]*bars
            limit=bars-InpDepth
        else:
            # set start position to found extremum position
            limit=i-1;

            # what kind of extremum?
            if ExtLowBuffer[i]!=0: 
                # low extremum
                curlow=ExtLowBuffer[i]
                # will look for the next high extremum
                whatlookfor=1
            else:
                # high extremum
                curhigh=ExtHighBuffer[i]
                # will look for the next low extremum
                whatlookfor=-1

            # clear the rest data
            for i in range(limit-1, -1, -1):
                ExtZigzagBuffer[i]=0 
                ExtLowBuffer[i]=0
                ExtHighBuffer[i]=0

    # main loop 
    for i in range(limit, -1, -1): 
        # find lowest low in depth of bars
        extremum=min(low[i:i+InpDepth])
        # this lowest has been found previously
        if extremum==lastlow: 
            extremum=0
        else: 
            # new last low
            lastlow=extremum
            # discard extremum if current low is too high
            if low[i]-extremum>InpDeviation*point: 
                extremum=0
            else: 
                # clear previous extremums in backstep bars
                for back in range(1, InpBackstep+1): 
                    pos=i+back
                    if ExtLowBuffer[pos]!=0 and ExtLowBuffer[pos]>extremum:
                        ExtLowBuffer[pos]=0

        # found extremum is current low
        if low[i]==extremum: 
            ExtLowBuffer[i]=extremum
        else: 
            ExtLowBuffer[i]=0

        # find highest high in depth of bars
        extremum=max(high[i:i+InpDepth])
        # this highest has been found previously
        if extremum==lasthigh:
            extremum=0
        else: 
            # new last high
            lasthigh=extremum
            # discard extremum if current high is too low
            if extremum-high[i]>InpDeviation*point: 
                extremum=0
            else: 
                # clear previous extremums in backstep bars
                for back in range(1,InpBackstep+1): 
                    pos=i+back
                    if ExtHighBuffer[pos]!=0 and ExtHighBuffer[pos]<extremum:
                        ExtHighBuffer[pos]=0

        # found extremum is current high
        if high[i]==extremum: 
            ExtHighBuffer[i]=extremum
        else: 
            ExtHighBuffer[i]=0

    # final cutting 
    if whatlookfor==0: 
        lastlow=0
        lasthigh=0
    else: 
        lastlow=curlow
        lasthigh=curhigh
    for i in range(limit, -1, -1): 
        if whatlookfor==0: # look for peak or lawn
            if lastlow==0 and lasthigh==0 and ExtHighBuffer[i]!=0:
                lasthigh=high[i]
                lasthighpos=i
                whatlookfor=-1
                ExtZigzagBuffer[i]=lasthigh
            if lastlow==0 and lasthigh==0 and ExtLowBuffer[i]!=0: 
                lastlow=low[i]
                lastlowpos=i
                whatlookfor=1
                ExtZigzagBuffer[i]=lastlow
            pass

        if whatlookfor==1: # look for peak
            if ExtLowBuffer[i]!=0 and ExtLowBuffer[i]<lastlow and ExtHighBuffer[i]==0:
                ExtZigzagBuffer[lastlowpos]=0
                lastlowpos=i
                lastlow=ExtLowBuffer[i]
                ExtZigzagBuffer[i]=lastlow
            if ExtHighBuffer[i]!=0 and ExtLowBuffer[i]==0:
                lasthigh=ExtHighBuffer[i]
                lasthighpos=i
                ExtZigzagBuffer[i]=lasthigh
                whatlookfor=-1
            pass

        if whatlookfor==-1: # look for lawn 
            if ExtHighBuffer[i]!=0 and ExtHighBuffer[i]>lasthigh and ExtLowBuffer[i]==0: 
                ExtZigzagBuffer[lasthighpos]=0
                lasthighpos=i
                lasthigh=ExtHighBuffer[i]
                ExtZigzagBuffer[i]=lasthigh
            if ExtLowBuffer[i]!=0 and ExtHighBuffer[i]==0:
                lastlow=ExtLowBuffer[i]
                lastlowpos=i
                ExtZigzagBuffer[i]=lastlow
                whatlookfor=1
            pass

    #ExtZigzagBuffer = [np.nan if x == 0 else x for x in ExtZigzagBuffer]

    return ExtHighBuffer, ExtLowBuffer      
  

In [3]:
def get_zzDone(df, InpDepth, InpDeviation, InpBackstep, pip ): 
    
    zzDir = 'zzDir' + str(InpDepth)
    zzValue = 'zzValue' + str(InpDepth)
    zz = 'zz' + str(InpDepth)
    
    df['zzHigh'], df['zzLow'] = get_zigzag(df, InpDepth, InpDeviation, InpBackstep, pip)
    
    def zz_dir (row):
        if row['zzLow'] != 0:
            return 'low'
        if row['zzHigh'] != 0: 
            return 'high' 
    
    def zz_value (row):
        if row['zzLow'] != 0:
            return row['zzLow']
        if row['zzHigh'] != 0: 
            return row['zzHigh']  
        
    df[zzDir] = df.apply (lambda r : zz_dir(r), axis=1)
    df[zzValue] = df.apply (lambda r : zz_value(r), axis=1)
    
    
    #remove not necessary peak/low and calculate percentage
    for i in range(len(df)): 
        if df[zzDir].iloc[i] != None:
            tempDir = df[zzDir].iloc[i]
            pos = i 
            for k in range(i+1, len(df)):
                if df[zzDir].iloc[k] == tempDir: 
                    df[zzDir].iloc[k] = None
                    df[zzValue].iloc[k] = np.nan
                    continue
                elif df[zzDir].iloc[k] != None: 
                    tempDir = df[zzDir].iloc[k]
                    pos = k
                    break
                i=k+1
        else: 
            pass

    df[zz] = df[zzValue].interpolate(method='linear')
    #df['zz'] = df.zz.replace(0, np.nan)
    
    df = df.drop(['zzHigh', 'zzLow'], axis=1)
    
    return df

### Plot zigzag points and moving average 

In [152]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime

import matplotlib.dates as mdates
from matplotlib.dates import MONDAY, DateFormatter, DayLocator, WeekdayLocator
from datetime import timedelta
from mplfinance.original_flavor import candlestick2_ohlc, candlestick_ohlc
import matplotlib.ticker as ticker

import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go

def plot_zz_cp(df_price,title,frequency,line1,line2,line3,line4):

    # tick for daily data
    mondays = WeekdayLocator(MONDAY)        
    alldays = DayLocator()              
    weekFormatter = DateFormatter('%b %d')  

    # tick for hourly data
    date_fmt = mdates.DateFormatter('%d/%b')
    hour_fmt = mdates.DateFormatter('%H:%M')

    fig, ax = plt.subplots()
    fig.subplots_adjust(bottom=0.5)

    if 'd' in frequency or 'D' in frequency: 
        ax.xaxis.set_major_locator(mondays)
        ax.xaxis.set_minor_locator(alldays)
        ax.xaxis.set_major_formatter(weekFormatter)    
        w=0.6
    else: 
        ax.xaxis.set_major_locator(mdates.DayLocator(interval=1))
        ax.xaxis.set_major_formatter(date_fmt)
        ax.xaxis.set_minor_locator(mdates.HourLocator(interval=4))
        ax.xaxis.set_minor_formatter(hour_fmt)
        ax.xaxis.set_tick_params(which='major', pad=15) 
        w=0.006 

    candlestick_ohlc(ax, zip(mdates.date2num(df_price.index.to_pydatetime()),
                       df_price['open'], df_price['high'],
                       df_price['low'], df_price['close']),
               width=w,colordown='pink',colorup='lightgreen')


    ax.xaxis_date()
    ax.autoscale_view()
    fig.set_size_inches(25,20)
    plt.setp(plt.gca().get_xticklabels(), rotation=45, horizontalalignment='right')
    plt.xticks(rotation=45)
    ax.set_title(title)
    ymin, ymax = ax.get_ylim()
    #ax.set_ylim([0.5,1.2])

    plt.plot(df_price['zzValue84'], color='olive', linestyle='None', marker='o', markersize=12)

    if line1 == '': 
        plt.plot(df_price['zz84'], color='olive', label='zz', linestyle='dotted', linewidth=2) #, marker='o', markersize=12)
    if line1 != '':
        plt.plot(df_price[line1], color='black', label=line1, linewidth=2.5)
    if line2 != '':
        plt.plot(df_price[line2], color='cyan', label=line2, linestyle='dashed', linewidth=2)
    if line3 != '':
        plt.plot(df_price[line3], color='purple', label=line3, linestyle='dashed', linewidth=2)
    if line4 != '':
        plt.plot(df_price[line4], color='red', label=line4, linestyle='dashed', linewidth=2)

        plt.legend(loc='best')

    plt.show()

In [153]:
df = pd.read_csv("EURUSD_H1_202209230000_202210050700.csv")
df.columns=['data','time', 'open', 'high', 'low', 'close']
df.reset_index(drop=True, inplace=True)
df.isna().sum()
df.head()

,data,time,open,high,low,close
0,2022.09.23,0:00:00,0.98283,0.98400,0.98244,0.98362
1,2022.09.23,1:00:00,0.98362,0.98386,0.98315,0.98319
2,2022.09.23,2:00:00,0.98319,0.98398,0.98317,0.98383
3,2022.09.23,3:00:00,0.98383,0.98485,0.98324,0.98432
4,2022.09.23,4:00:00,0.98434,0.98520,0.98320,0.98373


In [154]:
a = get_zigzag(df, 7,5,3, 0.0001)
para_a = '7,5,3'

In [155]:
b = get_zigzag(df, 114,10,3, 0.0001)

In [156]:
def pivot_points(df,zigzag, parameter):
    df['value'+(parameter)] = np.zeros(len(df))
    df[parameter] = np.zeros(len(df))
    high = zigzag[0]
    low = zigzag[1]
    for i in df.high[np.array(high)!=0].index:
        df.loc[i,[parameter]]= 1
        df.loc[i,['value'+(parameter)]]=df.high[i]
    for i in df.low[np.array(low)!=0].index:
        df.loc[i,[parameter]]=-1
        df.loc[i,['value'+(parameter)]]=df.low[i]

In [157]:
pivot_points(df,a,para_a)

In [166]:
df['value'+para_a]

0      0.0000
1      0.0000
2      0.0000
3      0.0000
4      0.9852
        ...  
195    0.0000
196    0.0000
197    0.0000
198    0.0000
199    0.0000
Name: value7,5,3, Length: 200, dtype: float64

In [168]:
dfpl = df[:]
ts_pivots = pd.Series(dfpl['value'+para_a], index=dfpl['value'+para_a].index)
ts_pivots = ts_pivots[df[para_a]!=0]

In [169]:
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])
fig.add_trace(go.Scatter(x=ts_pivots.index, y=ts_pivots.values,line_color='blue'))
fig.show()